In [1]:
import laspy
import scipy
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.filters import gabor_kernel
from scipy import ndimage as ndi

import matplotlib.pyplot as plt
from numba import jit
import pickle
import os

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
from IPython.display import clear_output

C:\Users\nivan\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
class survey:

    def __init__(self, filepath, parampath, outputpath):

        if filepath.endswith('/'):
            self.filepath = filepath
        else:
            self.filepath = filepath + '/'

        self.parampath = parampath

        if outputpath.endswith('/'):
            self.outputpath = outputpath
        else:
            self.outputpath = outputpath + '/'
            
        self.features = None

    def create_dir(self):

        binsize = self.parampath["resolution"]
        folders_main = self.outputpath
        survey_name = self.parampath["survey_name"]

        raw = 'raw'
        resolution = 'resolution'
        h = '1_histogram'
        txtr = '2_rawtexturefeatures'
        PCfeat = '3_PCfeatures'
        clfs = '4_classifiers'
        res_value = str(binsize)

        survey_folder = folders_main + survey_name

        raw_survey = survey_folder + '/' + raw
        res_survey = survey_folder + '/' + resolution + res_value

        hist = res_survey + '/' + h
        txtfeat = res_survey + '/' + txtr

        allfolders = (survey_folder, raw_survey, res_survey, hist, txtfeat)

        for elem in allfolders:
            try:
                os.mkdir(elem)
            except:
                continue

##################################################################################

    # @jit(parallel=True)
    def downsample(self):

        inFile = laspy.file.File(self.filepath, mode='r')
        # load data attributes
        x = inFile.x
        y = inFile.y
        z = inFile.z
        Insty = inFile.intensity
        labels = inFile.raw_classification
        binsize = self.parampath["resolution"]

        # initialize arrays
        xbin = np.arange(np.floor(x.min()), np.ceil(x.max()), binsize)
        ybin = np.arange(np.floor(y.min()), np.ceil(y.max()), binsize)
        count = np.zeros((xbin.shape[0], ybin.shape[0]), dtype=np.uint16)
        zsum = np.zeros((xbin.shape[0], ybin.shape[0]))
        Itsum = np.zeros((xbin.shape[0], ybin.shape[0]))
        labelsum = np.zeros((xbin.shape[0], ybin.shape[0]))

        # bin data
        for i in range(len(x)):
            if ~(labels[i] == 7) & ~(labels[i] == 18):  # ignore noise values
                xi = np.digitize(x[i], xbin)
                yi = np.digitize(y[i], ybin)
                count[xi-1, yi-1] += 1
                zsum[xi-1, yi-1] += z[i]
                Itsum[xi-1, yi-1] += Insty[i]
                labelsum[xi-1, yi-1] += labels[i]
            else:
                continue
            clear_output(wait = True)
            print('Downsampling: ', (i/len(x)) * 100, '%',flush = True)

        np.seterr(invalid='ignore')

        x, y = np.meshgrid(ybin, xbin)
        z = zsum / count
        I = Itsum / count
        labels = labelsum / count

        # original labels: water is 9, unclassified is 0, reef is 26
        water = labels >= 4.5
        unclass = labels < 4.5
        labels[unclass] = 1
        labels[water] = 2

        histpath = self.outputpath + self.parampath["survey_name"] + '/resolution' + str(
            self.parampath["resolution"]) + '/1_histogram/' + 'hist' + '.npz'

        np.savez(histpath, x=x, y=y, count=count, z=z, I=I, labels=labels)


    def gaborkernels(self):

        parameters = self.parampath
        num_orientations = self.parampath["gabor_orient"]
        bandwidths = self.parampath["gabor_bw"]
        frequencies = self.parampath["gabor_freq"]

        kernels = []

        # loop through kernel orientations
        for theta in range(int(num_orientations)):
            theta = theta / num_orientations * np.pi

            # loop through bandwidths
            for sigma in bandwidths:

                # loop through frequencies
                for frequency in frequencies:

                    # calculate and take the real part of a gabor wavelet
                    kernel = np.real(gabor_kernel(frequency, theta=theta,
                                                  sigma_x=sigma, sigma_y=sigma))

                    # append to kernel list
                    kernels.append(kernel)

        return kernels

    def compute_feats(self, image, kernels):
        # """Function to calculate the features of an image using a bank of convolution kernels.

        # The function takes two inputs.
        # 'image' is a 2D numpy array.
        # 'kernels' is a list of 2D numpy arrays

        # The output will be a numpy array of shape (len(kernels) x 2) where the first column
        # contains the mean of each filtered image, and the second column contains the
        # variance of each filtered image."""

        # replace nans with zeros
        imagenan = np.isnan(image)
        image[imagenan] = 0

        # create the feaeture array, one column for mean, and one for variance.
        feats = np.zeros((2, len(kernels)), dtype=np.double)

        # loop through the gabor wavelet kernels
        for k, kernel in enumerate(kernels):

            # filter the image using the gabor wavelet
            filtered = ndi.convolve(image, kernel, mode='wrap')

            # calculate mean and variance of the filtered image, and add each to the feature array
            feats[0, k] = np.mean(filtered)
            feats[1, k] = np.var(filtered)

        return feats

    @jit
    def blockprocess(self, hists):

        parameters = self.parampath

        attr_name = parameters["texture_attr"]
        step = parameters["step_size"]
        block_size = parameters["block_size"]

        if attr_name == 'intensity':
            attr = hists['I']
        elif attr_name == 'height':
            attr = hists['z']
        else:
            raise Exception(
                "texture_attr must be 'intensity' or 'height'.")

        # half block size to get indices of block around a pixel.
        hb = int(block_size/2)

        # initialize numpy arrays for mean and var of each filtered block

        num_samples = np.floor(
            ((attr.shape[0]-(2*hb))/step))*np.floor(((attr.shape[1]-(2*hb))/step))

        kernels = self.gaborkernels()
        numkern = len(kernels)

        # kernels + x , y, z, I, labels
        attr_mean = np.zeros((num_samples.astype(int), numkern+5))
        # attr_var = np.zeros_like(attr_mean)

        # loop through pixels, starting at half a blocksize away from the edge ( to avoid padding)
        # and incremented by step.
        i = 0
        for xi in range(hb, attr.shape[0]-hb, step):
            for yi in range(hb, attr.shape[1]-hb, step):

                # if the block has no data in it, just assign the mean and var of that block to zero
                if np.isnan(attr[xi, yi]):
                    attr_mean[i, 5:] = np.nan

                    # attr_var[i, 5:] = np.nan

                # calculate features and add mean and var to corresponding numpy arrays.
                else:
                    coldfeat = self.compute_feats(
                        attr[xi-hb:xi+hb, yi-hb:yi+hb], kernels)

                    attr_mean[i, 5:] = coldfeat[0, :]
                    # attr_var[i, 5:] = coldfeat[1, :]

                attr_mean[i, 0] = hists['x'][xi, yi]
                attr_mean[i, 1] = hists['y'][xi, yi]
                attr_mean[i, 2] = hists['z'][xi, yi]
                attr_mean[i, 3] = hists['I'][xi, yi]
                attr_mean[i, 4] = hists['labels'][xi, yi]

                # attr_var[i, :5] = attr_mean[i, numkern:]

                i += 1

        return attr_mean

    def texture_features(self):

        parameters = self.parampath

        histspath = self.outputpath + self.parampath["survey_name"] + '/resolution' + str(
            self.parampath["resolution"]) + '/1_histogram/' + 'hist' + '.npz'

        hists = np.load(histspath)

        texturefeats = self.blockprocess(hists)

        outfile = self.outputpath + self.parampath["survey_name"] + '/resolution' + \
            str(parameters["resolution"]) + '/2_rawtexturefeatures/' + \
            'hist' + '_res' + \
            str(parameters["resolution"]) + '_raw_feats.npy'

        np.savetxt(outfile, texturefeats)

    def texture(self):

        ##########################################################

        num_orientations = self.parampath["gabor_orient"]
        bandwidths = self.parampath["gabor_bw"]
        frequencies = self.parampath["gabor_freq"]

        kernels = []

        # loop through kernel orientations
        for theta in range(int(num_orientations)):
            theta = theta / num_orientations * np.pi

            # loop through bandwidths
            for sigma in bandwidths:

                # loop through frequencies
                for frequency in frequencies:

                    # calculate and take the real part of a gabor wavelet
                    kernel = np.real(gabor_kernel(frequency, theta=theta,
                                                  sigma_x=sigma, sigma_y=sigma))

                    # append to kernel list
                    kernels.append(kernel)

        ###########################################################

        histspath = self.outputpath + self.parampath["survey_name"] + '/resolution' + str(
            self.parampath["resolution"]) + '/1_histogram/' + 'hist' + '.npz'

        hist = np.load(histspath)

        attr = hist['z']

        ###############################################################

        block_size = self.parampath["block_size"]
        step_size = self.parampath["step_size"]

        hb = int(block_size/2)

        ##################################################################

        num_samples = np.floor(
            ((attr.shape[0]-(2*hb))/step_size))*np.floor(((attr.shape[1]-(2*hb))/step_size))
        numkern = len(kernels)

        # kernels + x , y, z, I, labels

        features = np.zeros((num_samples.astype(int), numkern+5))

        ###################################################################
        i = 0
        for xi in range(hb, attr.shape[0]-hb, step_size):
            for yi in range(hb, attr.shape[1]-hb, step_size):

                if np.isnan(attr[xi, yi]):
                    features[i, :] = np.nan
                else:
                    block = attr[xi-hb:xi+hb, yi-hb:yi+hb]
                    features[i, [0, 1, 2, 3, 4]] = [hist['x'][xi, yi], hist['y'][xi, yi],
                                                    hist['z'][xi, yi], hist['I'][xi, yi], hist['labels'][xi, yi]]

                    for kernel in kernels:
                        #print('kernel shape is : ', kernel.shape)
                        #print('block shape is : ', block.shape)
                        feature = np.mean(ndi.convolve(
                            block, kernel, mode='wrap'))
                        for j in range(5, len(kernels) + 5):
                            features[i, j] = feature
                            
                            
                clear_output(wait = True)            
                print('percent complete: ', (i/num_samples) * 100, '%',flush = True)
                i += 1

        ########################################################################

        clean_features = features[~np.isnan(features).any(axis=1)]
        cf = clean_features[clean_features[:,4] != 0]
        
        self.features = cf
        
        txtrpath = self.outputpath + self.parampath["survey_name"] + '/resolution' + str(
            self.parampath["resolution"]) + '/2_rawtexturefeatures/' + 'features' + '.npy'

        np.save(txtrpath,cf)
        

In [ ]:
parameters = {
    "survey_name" : '20190327_00568_00622_NoFilters_Torrey_DelMar',
    "resolution" : 1,
    
    "gabor_orient" : 4,
    "gabor_bw" : (1,3),
    "gabor_freq" : (0.125, 0.25),
    
    "block_size" : 20,
    "step_size" : 5,
    "texture_attr" : 'height' # or 'intensity'
}

lasfile = 'C:/Users/nivan/Documents/CCS_data/20190327_00568_00622_NoFilters_Torrey_DelMar.las'

outputdir = 'C:/Users/nivan/Desktop/Beach 2020/'

In [ ]:
survey_tdm = survey(lasfile,parameters,outputdir)

In [ ]:
survey_tdm.create_dir()

In [ ]:
features = survey_tdm.texture()

In [3]:
parameters = {
    "survey_name" : '20191209_00568_00636_2304_NoFilters_DelMar',
    "resolution" : 1,
    
    "gabor_orient" : 4,
    "gabor_bw" : (1,3),
    "gabor_freq" : (0.125, 0.25),
    
    "block_size" : 20,
    "step_size" : 5,
    "texture_attr" : 'height' # or 'intensity'
}

lasfile = 'C:/Users/nivan/Documents/CCS_data/20191209_00568_00636_2304_NoFilters_DelMar.las'

outputdir = 'C:/Users/nivan/Desktop/Beach 2020/'

In [ ]:
survey_dm = survey(lasfile,parameters,outputdir)
survey_dm.create_dir()
survey_dm.downsample()
survey_dm.texture()

Downsampling:  0.09988781929849344 %
